In [ ]:
# Weights & Biases Setup
# =====================

# Login to wandb (run this cell first)
!wandb login

# Optional: Set your wandb entity (username/team)
# !wandb login --relogin

In [ ]:
import os
import random

import numpy as np
import pandas as pd
import torch

# Import our custom modules
from config import Config
from dataset import create_data_loaders
from metrics import plot_roc_curves, plot_precision_recall_curves
from model import create_model, setup_model_for_training, count_parameters
from trainer import Trainer


def set_seed(seed: int) -> None:
    """Set random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# Set random seed for reproducibility
set_seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

print(f"CUDA Version: {torch.version.cuda}")
print(f"PyTorch Version: {torch.__version__}")

In [ ]:
# Create configuration object
config = Config()

print("Configuration:")
print(f"  Model: {config.model_name}")
print(f"  Learning Rate: {config.learning_rate}")
print(f"  Epochs: {config.epochs}")
print(f"  Batch Size: {config.batch_size}")
print(f"  Image Size: {config.img_size}")
print(f"  Device: {device}")
print(f"  Early Stopping Patience: {config.early_stopping_patience}")
print(f"  Early Stopping Min Delta: {config.early_stopping_min_delta}")
print(f"  LR Reduce Patience: {config.lr_reduce_patience}")
print(f"  LR Reduce Factor: {config.lr_reduce_factor}")
print(f"  LR Reduce Min LR: {config.lr_reduce_min_lr}")

In [ ]:
# Wandb Configuration for Hyperparameter Sweeping
# ===============================================

# Uncomment and modify these parameters for hyperparameter sweeping
# The sweep will automatically override these values during optimization

# config.model_name = 'convnextv2_tiny'  # Options: nano, tiny, base, large, huge
# config.learning_rate = 1e-5  # Range: 1e-6 to 1e-3
# config.batch_size = 32  # Options: 16, 32, 64
# config.img_size = 224  # Options: 224, 256, 384
# config.threshold = 0.5  # Range: 0.3 to 0.7
# config.early_stopping_patience = 10  # Options: 5, 10, 15
# config.lr_reduce_patience = 5  # Options: 3, 5, 7
# config.lr_reduce_factor = 0.5  # Options: 0.3, 0.5, 0.7

# Enable wandb logging
config.use_wandb = True
config.wandb_project = 'convnextv2-multilabel-classification'
config.wandb_entity = None  # Set to your wandb username/team
config.wandb_run_name = None  # Will be auto-generated
config.wandb_tags = ['multilabel', 'convnextv2', 'finetuning']

print("Wandb Configuration:")
print(f"  Enabled: {config.use_wandb}")
print(f"  Project: {config.wandb_project}")
print(f"  Entity: {config.wandb_entity}")
print(f"  Tags: {config.wandb_tags}")


In [ ]:
# Data Loading and Preparation
# ============================

print("Loading and preparing dataset...")

# Create data loaders with stratified splitting
train_loader, val_loader, label_columns = create_data_loaders(config)

print(f"Dataset loaded successfully!")
print(f"  Number of classes: {len(label_columns)}")
print(f"  Class names: {label_columns}")
print(f"  Training samples: {len(train_loader.dataset)}")
print(f"  Validation samples: {len(val_loader.dataset)}")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")

# Display sample batch info
sample_batch = next(iter(train_loader))
images, labels = sample_batch
print(f"\nSample batch shape:")
print(f"  Images: {images.shape}")
print(f"  Labels: {labels.shape}")
print(f"  Label range: [{labels.min():.3f}, {labels.max():.3f}]")
print(f"  Positive labels per sample: {labels.sum(dim=1).float().mean():.2f}")


In [ ]:
# Model Creation and Setup
# =========================

print("Creating and setting up model...")

# Create model
model = create_model(config, num_classes=len(label_columns))

# Setup model for training (freeze/unfreeze based on training mode)
model = setup_model_for_training(model, config)

# Count parameters
param_counts = count_parameters(model)
print(f"Model created successfully!")
print(f"  Total parameters: {param_counts['total']:,}")
print(f"  Trainable parameters: {param_counts['trainable']:,}")
print(f"  Frozen parameters: {param_counts['frozen']:,}")
print(f"  Training mode: Finetuning (backbone frozen, classifier trainable)")

# Move model to device
model = model.to(device)
print(f"  Model moved to: {device}")

# Test forward pass
print("\nTesting forward pass...")
with torch.no_grad():
    sample_images = torch.randn(2, 3, config.img_size, config.img_size).to(device)
    sample_output = model(sample_images)
    print(f"  Input shape: {sample_images.shape}")
    print(f"  Output shape: {sample_output.shape}")
    print(f"  Output range: [{sample_output.min():.3f}, {sample_output.max():.3f}]")


In [ ]:
# Training Setup
# ==============

print("Setting up trainer...")

# Create trainer
trainer = Trainer(
    model=model,
    config=config,
    train_loader=train_loader,
    val_loader=val_loader,
    label_columns=label_columns,
    device=device
)

print("Trainer setup complete!")
print(f"  Loss function: Focal Loss")
print(f"  Optimizer: AdamW (lr={config.learning_rate})")
print(f"  Scheduler: ReduceLROnPlateau (patience={config.lr_reduce_patience})")
print(f"  Early stopping: Enabled (patience={config.early_stopping_patience})")
print(f"  Mixed precision: Enabled")
print(f"  Threshold: {config.threshold}")
print(f"  Training mode: Finetuning")


In [ ]:
# Training Execution
# ==================

print("Starting training...")
print("=" * 50)

# Start training
history = trainer.train()

print("=" * 50)
print("Training completed!")
print(f"Best validation F1 Micro: {trainer.best_val_f1:.4f}")

# Plot training history
print("\nPlotting training history...")
trainer.plot_training_history(save_path=os.path.join(config.output_dir, 'training_history.png'))


In [ ]:
# Model Evaluation and Analysis
# =============================

print("Evaluating model on validation set...")

# Get final validation metrics
val_loss, val_metrics = trainer.validate_epoch()

print("\nFinal Validation Metrics:")
print(f"  Loss: {val_loss:.4f}")
print(f"  F1 Micro: {val_metrics['f1_micro']:.4f}")
print(f"  F1 Macro: {val_metrics['f1_macro']:.4f}")
print(f"  F1 Samples: {val_metrics['f1_samples']:.4f}")
print(f"  Precision Micro: {val_metrics['precision_micro']:.4f}")
print(f"  Precision Macro: {val_metrics['precision_macro']:.4f}")
print(f"  Recall Micro: {val_metrics['recall_micro']:.4f}")
print(f"  Recall Macro: {val_metrics['recall_macro']:.4f}")

if 'roc_auc_micro' in val_metrics:
    print(f"  ROC AUC Micro: {val_metrics['roc_auc_micro']:.4f}")
    print(f"  ROC AUC Macro: {val_metrics['roc_auc_macro']:.4f}")
    print(f"  PR AUC Micro: {val_metrics['pr_auc_micro']:.4f}")
    print(f"  PR AUC Macro: {val_metrics['pr_auc_macro']:.4f}")

# Get per-class metrics
print("\nComputing per-class metrics...")
per_class_metrics = trainer.metrics_calculator.compute_per_class_metrics(
    y_true=val_loader.dataset.labels,
    y_pred=np.concatenate([trainer.model(torch.tensor(batch[0]).to(device)).cpu().detach().numpy()
                           for batch in val_loader]),
    class_names=label_columns
)

print("\nPer-class F1 Scores:")
for class_name, metrics in per_class_metrics.items():
    print(f"  {class_name}: {metrics['f1']:.4f} (support: {metrics['support']})")


In [ ]:
# Visualization and Analysis
# ===========================

print("Generating visualizations...")

# Get predictions for visualization
trainer.model.eval()
all_predictions = []
all_labels = []
all_probabilities = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = trainer.model(images)
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        predictions = outputs.cpu().numpy()
        labels_np = labels.cpu().numpy()

        all_predictions.append(predictions)
        all_labels.append(labels_np)
        all_probabilities.append(probabilities)

all_predictions = np.concatenate(all_predictions, axis=0)
all_labels = np.concatenate(all_labels, axis=0)
all_probabilities = np.concatenate(all_probabilities, axis=0)

# Plot ROC curves
print("Plotting ROC curves...")
plot_roc_curves(
    all_labels,
    all_probabilities,
    label_columns,
    save_path=os.path.join(config.output_dir, 'roc_curves.png')
)

# Plot Precision-Recall curves
print("Plotting Precision-Recall curves...")
plot_precision_recall_curves(
    all_labels,
    all_probabilities,
    label_columns,
    save_path=os.path.join(config.output_dir, 'pr_curves.png')
)


In [ ]:
# Model Saving
# ============

print("Saving model and results...")

# Create output directory
os.makedirs(config.output_dir, exist_ok=True)

# Save model
model_path = os.path.join(config.output_dir, f'{config.model_name}_finetuned_best.pth')
trainer.save_model(model_path)

# Save training history as CSV
history_df = pd.DataFrame(history)
history_path = os.path.join(config.output_dir, 'training_history.csv')
history_df.to_csv(history_path, index=False)
print(f"Training history saved to: {history_path}")

# Save configuration
config_path = os.path.join(config.output_dir, 'config.json')
import json

with open(config_path, 'w') as f:
    config_dict = {
        'model_name': config.model_name,
        'training_mode': 'finetuning',
        'learning_rate': config.learning_rate,
        'epochs': config.epochs,
        'batch_size': config.batch_size,
        'img_size': config.img_size,
        'threshold': config.threshold,
        'label_columns': label_columns,
        'best_val_f1': trainer.best_val_f1
    }
    json.dump(config_dict, f, indent=2)
print(f"Configuration saved to: {config_path}")

print(f"\nAll files saved to: {config.output_dir}")
print("Training completed successfully!")


In [ ]:
# Weights & Biases Hyperparameter Sweep
# =====================================

print("Wandb Sweep Options:")
print("\n1. Manual Sweep (Run this notebook multiple times with different configs)")
print("   - Modify the config parameters in cell 2")
print("   - Run all cells to train with new parameters")
print("   - Compare results in wandb dashboard")

print("\n2. Automated Sweep (Recommended)")
print("   - Run: python run_sweep.py")
print("   - This will automatically test different hyperparameter combinations")
print("   - Uses Bayesian optimization to find best parameters")

print("\n3. Sweep Configuration:")
print("   - Model: convnextv2_nano, convnextv2_tiny, convnextv2_base")
print("   - Learning Rate: 1e-6 to 1e-3 (log scale)")
print("   - Batch Size: 16, 32, 64")
print("   - Image Size: 224, 256, 384")
print("   - Threshold: 0.3, 0.4, 0.5, 0.6, 0.7")
print("   - Early Stopping Patience: 5, 10, 15")
print("   - LR Reduce Patience: 3, 5, 7")
print("   - LR Reduce Factor: 0.3, 0.5, 0.7")

print("\n4. Key Metrics Tracked:")
print("   - val_f1_micro (primary metric for optimization)")
print("   - val_f1_macro, val_f1_samples")
print("   - val_precision_micro, val_precision_macro")
print("   - val_recall_micro, val_recall_macro")
print("   - val_roc_auc_micro, val_roc_auc_macro")
print("   - val_pr_auc_micro, val_pr_auc_macro")
print("   - train_loss, val_loss")
print("   - learning_rate, best_val_f1")

print("\n5. To start a sweep:")
print("   !python run_sweep.py")
print("   # Or run the sweep configuration directly:")
print("   !wandb sweep wandb_sweep_config.yaml")


In [ ]:
# Early Stopping and Learning Rate Reduction
# ===========================================

print("Training Features:")
print("\n1. Early Stopping:")
print(f"   - Patience: {config.early_stopping_patience} epochs")
print(f"   - Min Delta: {config.early_stopping_min_delta}")
print("   - Monitors: Validation F1 Micro score")
print("   - Stops training when no improvement for patience epochs")

print("\n2. Learning Rate Reduction on Plateau:")
print(f"   - Patience: {config.lr_reduce_patience} epochs")
print(f"   - Factor: {config.lr_reduce_factor} (reduces LR by this factor)")
print(f"   - Min LR: {config.lr_reduce_min_lr}")
print("   - Monitors: Validation F1 Micro score")
print("   - Reduces LR when no improvement for patience epochs")

print("\n3. Benefits:")
print("   - Prevents overfitting with early stopping")
print("   - Automatically adjusts learning rate for better convergence")
print("   - Saves training time by stopping when no improvement")
print("   - Always loads the best model at the end")

print("\n4. Monitoring:")
print("   - Watch for 'New best F1 Micro' messages")
print("   - Watch for 'Reducing learning rate' messages")
print("   - Training will stop early if no improvement")
print("   - Best model is automatically loaded at the end")


In [ ]:
# Model Comparison and Experimentation
# ====================================

print("Model comparison and experimentation options:")
print("\n1. Try different ConvNeXt V2 variants:")
print("   - convnextv2_nano: Fastest, smallest")
print("   - convnextv2_tiny: Good balance (current)")
print("   - convnextv2_base: Better accuracy")
print("   - convnextv2_large: High accuracy")
print("   - convnextv2_huge: Best accuracy, slowest")

print("\n2. Finetuning benefits:")
print("   - Fast training (only classifier head is trained)")
print("   - Good for small to medium datasets")
print("   - Preserves pretrained features")

print("\n3. Try different loss functions:")
print("   - Focal Loss: Good for class imbalance (current)")
print("   - Asymmetric Loss: Alternative for imbalance")
print("   - Weighted BCE: Simple weighted approach")

print("\n4. Hyperparameter tuning suggestions:")
print("   - Learning rate: 1e-5 to 1e-3")
print("   - Batch size: 16, 32, 64")
print("   - Image size: 224, 256, 384")
print("   - Threshold: 0.3 to 0.7")

print("\nTo experiment with different models, modify the config in cell 2:")
print("config.model_name = 'convnextv2_base'  # Change model")
print("config.epochs = 30  # Change epochs")
print("config.learning_rate = 2e-5  # Change learning rate")

print("\nTraining completed! Check the outputs directory for results.")
